In [14]:
from typing import List

In [15]:
def _compute_binary_relevance(
    recommended_items_list: List[int], #Это список из целых чисел. Каждое целое число - музыкальный трек, который модель порекомендовала послушать пользователю.
    true_items_list: List[int], #Это список из целых чисел. Каждое целое число - музыкальный трек, который пользователь реально послушал.
    ) -> List[int]: #Надо вернуть список из целых чисел такой же длины, как и recommended_items_list. Для каждого трека в списке recommended_items_list надо проставить 0 или 1. Это делается
  # на основе того, был ли рекомендованный трек реально прослушан. 1 - если рекомендованный трек был реально прослушан. 0 - иначе
  # your code here:

  answer=[]
  
  for i in range(0, len(recommended_items_list),1):

    if recommended_items_list[i] in true_items_list:
        answer.append(1)
    else:
        answer.append(0)
        
  return answer




In [16]:
a=[1,2,3,4]
b=[2,3]
c=[]

for i in range(0, len(a),1):
    if a[i] in b:
        c.append(1)
    else:
        c.append(0)

print(c)

[0, 1, 1, 0]


In [17]:
print(_compute_binary_relevance(a,b))

[0, 1, 1, 0]


In [18]:
def ap_at_k(
    recommended_items_list: List[int],
    true_items_list: List[int],
    k: int
) -> float:
  # your code here:

  #k — это количество рекомендаций, которые мы рассматриваем (первые k элементов в списке рекомендаций).

  k=min(k, len(recommended_items_list)) #Проверяем, чтобы k не оказалось больше кол-ва рекомендованных треков в recommended_items_list

  #Посчитаем p@k - precision at k.

  relevant=_compute_binary_relevance(recommended_items_list,true_items_list) #Список из нулей и единиц. Показывает угадала ли модель предложив тот или иной трек по сравнению с реально прослушанными треками

  p_at_k_list=[]


  for i in range(0,k,1):
    x=[relevant[j] for j in range(0,i+1,1)]
    l=len(x)
    a=sum(x)/l
    p_at_k_list.append(a)

  #Посчитаем P

  P=[]

  for i in range(0,k,1):
    if relevant[i]==1:
      P.append(p_at_k_list[i])
    else:
      P.append(0)

  m=len(set(true_items_list)) #Мощность множества реально прослушанных треков. Set чтобы не было повторных

  mini=min(k,m)

  if mini==0: #Чтобы не было деления на ноль
    return 0
  else:
    ap_at_k_otvet=sum(P)/mini
    return ap_at_k_otvet

  

In [19]:
relevant=[0,1,1,1]

p_at_k_list=[]

k=3

for i in range(0,k,1):
    x=[relevant[j] for j in range(0,i+1,1)]
    l=len(x)
    a=sum(x)/l
    p_at_k_list.append(a)

print(p_at_k_list)

[0.0, 0.5, 0.6666666666666666]


In [20]:
def map_at_k(
    recommended_items_lists: List[List[int]], #Это список СПИСКОВ!!! 
    true_items_lists: List[List[int]],
    k: int,
) -> float:
  """
  Computes ap@k for all buyers
  """
  assert len(recommended_items_lists) == len(true_items_lists), \
  'len(true_items_list) != len(recommended_items_list)'

  # your code here:

  N=len(true_items_lists) #Число пользователей

  ap_at_k_list=[ap_at_k(recommended_items_lists[i], true_items_lists[i],k) for i in range(0,N, 1)]

  map_at_k_otvet=sum(ap_at_k_list)/N

  return map_at_k_otvet




In [21]:
import numpy as np

def jaccard_sim(ratings: np.array, user_vector: np.array) -> np.array:
    """
    Computes jaccard similarities between user (represented by user_vector) and users in ratings matrix
    
    Args:
        ratings: matrix of ratings of shape (# users we want to find distance to) x n_items
        user_vector: user ratings vector
    Returns:
        jaccard_sim_arr: jaccard distances from this user to others 
            (this vector length should equals ratings.shape[0])
    """
    # your code here:
    ...
    
    #Входные данные выглядят к примеру так.
    #Каждая строка - какой-то пользователь.
    #Каждый столбец - номер прослушанного трека. 1 - трек прослушан. 0 - не прослушан

    #ratings = [[1, 0, 1, 0],  # пользователь v1 
              # [0, 1, 1, 0],  # пользователь v2
              # [1, 1, 0, 1]]  # пользователь v3

    #Вектор прослушанных треков нашим юзером. 
    #user_vector = [1, 1, 0, 0]  # пользователь u

    #Пересечение множеств.

    power_per=[] #Список. Мощность (A пересечь B) с длиной равной кол-ву других юзеров.
    
    for i in range(0,ratings.shape[0],1): #Для каждого юзера 
        power_i=0
        for j in range (0,len(user_vector),1): #Для каждого элемента в векторе
            if user_vector[j]==ratings[i, j] and user_vector[j]==1:
                power_i+=1
        power_per.append(power_i)

    #Объединение множеств.

    power_ob=[]

    for i in range(0,ratings.shape[0],1): #Для каждого юзера
        power_i=0
        for j in range (0,len(user_vector),1): #Для каждого элемента в векторе
            if user_vector[j]==1 or ratings[i, j]==1:
                power_i+=1
        power_ob.append(power_i)

    jaccard_sim_arr=[]
    for i in range(0, ratings.shape[0],1):
        if power_ob[i]!=0:
            jaccard_sim_arr.append(power_per[i]/power_ob[i])
        else:
            jaccard_sim_arr.append(0)

    jaccard_sim_arr=np.array(jaccard_sim_arr)

    return jaccard_sim_arr




In [22]:
import numpy as np

ratings = np.array([[1, 0, 1, 0],  # пользователь v1 
               [0, 1, 1, 0],  # пользователь v2
              [1, 1, 0, 0]])  # пользователь v3

    #Вектор прослушанных треков нашим юзером. 
user_vector = [1, 1, 0, 0]  # пользователь u

    #Пересечение множеств.

power=[] #Список. Мощность (A пересечь B) с длиной равной кол-ву других юзеров.
    
for i in range(0,ratings.shape[0],1): #Для каждого юзера 
    power_i=0
    for j in range (0,len(user_vector),1): #Для каждого элемента в векторе
        if user_vector[j]==ratings[i, j] and user_vector[j]==1:
            power_i+=1
    power.append(power_i)

print(power)

power_ob=[]

for i in range(0,ratings.shape[0],1): #Для каждого юзера
    power_i=0
    for j in range (0,len(user_vector),1): #Для каждого элемента в векторе
        if user_vector[j]==1 or ratings[i, j]==1:
            power_i+=1
    power_ob.append(power_i)

print(power_ob)

[1, 1, 2]
[3, 3, 2]


In [23]:
a=[1,2,2]
print(set(a))

{1, 2}


In [24]:
class User2User(BaseModel):
    def __init__(self, ratings, similarity_func: similarityFuncType=jaccard_sim, alpha: float=0.02):
        super().__init__(ratings)

        self.similarity_func = similarity_func
        self.alpha = alpha

    def similarity(self, user_vector: NDArray[int]) -> NDArray[float]:
        """Computes similarities between user_vector and all vectors in self.R
        Args:
            user_vector: vector of ratings, user has given to all tracks
        Returns:
            vector of simillarities between this user and all users in self.R
        """
        # your code here:
        
        return self.similarity_func(self.R, user_vector)

NameError: name 'BaseModel' is not defined

In [ ]:
def get_items_scores(self, uid: int) -> NDArray[float]:
    """Computes scores \hat{r}_{ui} (https://www.overleaf.com/read/vcqdvjkbjgzf#9c30ef) for all items in rating matrix for
    a particular user uid. Don't forget, that similarity of user u with himself should
    be zero to not get recommendetations from his own interactions.

    Args:
        uid (int): index of user from rating matrix
    Returns:
        scores_u (NDArray[float]): array of scores for all items
    """
    # your code here:

    #self.R = [
    #[1, 0, 1, 0, ...],  # Пользователь 0: 1 = слушал трек, 0 = не слушал
    #[0, 1, 0, 1, ...],  # Пользователь 1
    #[1, 0, 0, 1, ...],  # Пользователь 2
    #...

    #Посчитаем вектор в котором будут меры Жаккара.
    jaccard = self.similarity(self.R[uid])
    jaccard[uid]=0 #Схожесть Жаккара нашего объекта uid с самим собой равна нулю по условию.

    #Определим множество соседей.

    jaccard_neighbours={} #{index_usera: мера Жаккара}

    for i in range(0,len(jaccard)):
        if jaccard[i]>self.alpha:
            jaccard_neighbours[i]=jaccard[i]
    
    #Знаменатель формулы
    znamenatel=0.0
    for i in jaccard_neighbours:
        znamenatel+=jaccard_neighbours[i]

    #Числитель для формулы

    chislitel=np.zeros(self.R.shape[1])
    r=np.zeros(self.R.shape[1])
    for j in range(0,self.R.shape[1],1): #Бежим по всем трекам
        for i in jaccard_neighbours: #По всем соседям идем
            chislitel[j]+=jaccard_neighbours[i]*self.R[i,j]
        if znamenatel!=0:
            r[j]=chislitel[j]/znamenatel
        else:
            r[j]=0


    return r


    

        


In [ ]:
a={1:2, 2:3}

for i in a:
    print(i)

1
2


In [ ]:
import numpy as np
import pandas as pd

def _als_user_step(
    items_embeddings: NDArray[float],
    user_ratings: NDArray[float],
    reg_coef: float,
) -> NDArray[float]:
    """
    ALS model (https://yadi.sk/i/7ZONA2kIqROfRQ) consists of 2 steps: 
        1) recompute users embeddings
        2) recompute items embeddings
    
    This function allows one to recompute embedding for one particular user,
    given ratings that he gave and items_embeddings of those items, that the user has rated
    """
    # V - items_embeddings
    # lamb - reg_coef
    # r - user_ratings

    lamb=reg_coef
    r=user_ratings
    V=items_embeddings
    V_t=V.T
    VV = V_t@V
    dim=VV.shape[0]
    I=np.eye(dim)

    u=np.linalg.inv((V_t @ V + lamb*I)) @ V_t @ r

    return u

NameError: name 'NDArray' is not defined

In [ ]:
def _als_item_step(
    users_embeddings: NDArray[float],
    items_ratings: NDArray[float],
    reg_coef: float,
) -> NDArray[float]:
    """
    ALS model (https://yadi.sk/i/7ZONA2kIqROfRQ) consists of 2 steps: 
        1) recompute users embeddings
        2) recompute items embeddings
    
    This function allows one to recompute embedding for one particular item,
    given ratings that we have for this item from different users and users_embeddings of those users, who have rated this item
    """
    # your code here
    
    lamb=reg_coef
    r=items_ratings
    U=users_embeddings
    U_t=U.T
    UU=U_t @ U
    dim=UU.shape[0]
    I=np.eye(dim)

    v = np.linalg.inv(U_t @ U + lamb*I)@U_t@r

    return v

In [29]:
# test_als.py
import numpy as np
import sys
sys.path.append('.')  # или путь к utils

from utils.models import _als_user_step

# Тестовые данные из задания
V = np.array([
                [0, 1.0, 0],
                [-1.0, 2.0, 2.1],
                [1.5, 1.1, 2.0],
            ])
r = np.array([5.0, 1.0, 2.0])
reg = 1.0

print("V dtype:", V.dtype)
print("r dtype:", r.dtype)

try:
    result = _als_user_step(V, r, reg)
    print("Success! Result:", result)
except Exception as e:
    print("Error:", e)
    import traceback
    traceback.print_exc()

V dtype: float64
r dtype: float64
Success! Result: [ 0.81186175  2.04128491 -0.81773635]


In [ ]:
np.array([0.81186175, 2.04128491, -0.81773635])

In [30]:
# test_als.py
import numpy as np
import sys
sys.path.append('.')  # или путь к utils

from utils.models import _als_user_step

# Тестовые данные из задания
V = np.array([
                [0, 1.0, 0, 1.0, 1.5, -1.5, 0],
                [-1.0, 2.0, 2.1, 3.1, 3.1, -2.0, 1.1],
                [1.5, 1.1, 2.0, 11.5, -3.1, 5.7, -0.5],
            ])
r = np.array([5.0, 1.0, 2.0])
reg = 2.0

print("V dtype:", V.dtype)
print("r dtype:", r.dtype)

try:
    result = _als_user_step(V, r, reg)
    print("Success! Result:", result)
except Exception as e:
    print("Error:", e)
    import traceback
    traceback.print_exc()

V dtype: float64
r dtype: float64
Success! Result: [ 0.55809731  0.43001109 -0.8480172   0.57417105  0.29982033 -0.64635934
 -0.54156039]


In [ ]:
np.array([0.55809731, 0.43001109, -0.8480172, 0.57417105, 0.29982033, -0.64635934, -0.54156039])

In [ ]:
class ALS:
    """
    Class for training LFM model with ALS algorithm
    (formula (1.2) from this file: https://yadi.sk/i/7ZONA2kIqROfRQ)
    NOTE: in this inplementation we won't train user and item biases, thus the optimisation
        problem looks like that:
        
        ∑_{(u,i): \exists r_{ui}} (r_{ui} - <p_{u}, q_{i}>)^2 + reg_coef/2 * (∑_{u} ||p_{u}||^2 + ∑_{i} ||q_{i}||^2)  -> min_{P, Q}
    """

    def __init__(
        self,
        embeddings_dim: int=16,
        reg_coef: float=1.0,
        random_seed: int=59812
    ):
        self.embeddings_dim = embeddings_dim
        self.random_seed = random_seed
        self.reg_coef = reg_coef
        self.user2emb, self.item2emb = {}, {}
        self.users_embeddings: Optional[EmbeddingType] = None
        self.items_embeddings: Optional[EmbeddingType] = None
        
    def fit(
        self, 
        interactions: pd.DataFrame, 
        epochs: int=20, 
        verbose: bool=False,
        embeddings_initialized: bool=False,
    ):
        """
        Trains the model - iteratively recomputes self.users_embeddings and self.items_embeddings
        
        Args:
            interactions: dataframe of interactions (necessary columns: user_col, item_col, rating_col)
                for model training
            epochs: amount of iterations to recompute users_embeddings and items_embeddings
            verbose: whether to do additional logging during training or not
            embeddings_initialized (bool): whether to initialize self.users_embeddings and self.items_embeddings or
                they are already initialized
                
        IMPORTANT NOTE: first recompute users embeddings, then items embeddings
        """
        # your code here

        #userId  trackId  rating
        #    0       1     101     5.0    
        #    1       1     103     3.0
        #    2       2     102     4.0

        #interactions - первый столбец ID юзера, второй - ID трека, третий - оценка трека этим юзером. 

        return self